In [1]:
pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.1/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dgl
import json
import torch
import pickle
import numpy as np

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [3]:
with open("/kaggle/input/graph-base-v2-full/data_1000.pickle", "rb") as f:
    X, y = pickle.load(f)

In [4]:
def resize_tensor(tensor, target_length=128, pad_value=0):
    if tensor.size(1) > target_length:
        # Truncate the tensor if it's longer than the target length
        resized_tensor = tensor[:, :target_length]
    elif tensor.size(1) < target_length:
        # Pad the tensor if it's shorter than the target length
        pad_size = target_length - tensor.size(1)
        padding = torch.full((tensor.size(0), pad_size), pad_value, dtype=tensor.dtype)
        resized_tensor = torch.cat((tensor, padding), dim=1)
    else:
        # If the tensor is already the target length, return it as is
        resized_tensor = tensor

    return resized_tensor

In [5]:
def resize_array(arr, target_length=128, pad_value=0):
    if arr.shape[1] > target_length:
        # Truncate the array if it's longer than the target length
        return arr[:, :target_length]
    elif arr.shape[1] < target_length:
        # Pad the array if it's shorter than the target length
        pad_width = target_length - arr.shape[1]
        padding = np.full((arr.shape[0], pad_width), pad_value)
        return np.hstack((arr, padding))
    else:
        # If the array is already the target length, return it as is
        return arr

In [6]:
graphs = []
for i in range(len(X)):
    x = X[i]
    newx = np.array([resize_tensor(i) for i in x])
    newx = torch.tensor(newx)
#     print(newx.shape)
    g = dgl.DGLGraph()
    num_nodes = newx.shape[0]
    g.add_nodes(num_nodes)
    g.ndata['embedding'] = newx
    src, dst = zip(*y[0])
    g.add_edges(src, dst)
    graphs.append(g)

/opt/conda/lib/python3.10/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [7]:
len(graphs)

1500

In [8]:
graphs[0]

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'embedding': Scheme(shape=(1, 128), dtype=torch.int64)}
      edata_schemes={})

In [9]:
with open("graphs.picke", "wb") as f:
    pickle.dump(graphs, f)

In [10]:
with open("graphs.picke", "rb") as f:
    gs = pickle.load(f)
gs[0]

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'embedding': Scheme(shape=(1, 128), dtype=torch.int64)}
      edata_schemes={})